In [1]:
import numpy as np
import json

In [2]:
#vocabulary generate kore pura training corpus er

def generate_vocab(json_file_path, threshold=2):
    with open(json_file_path, "r") as json_file:
        json_data = json.load(json_file)

    vocab = {'<unk>': 0}
    counter = 0

    # Counting the number of times each word occurs
    for entry in json_data:
        sentence = entry["sentence"]
        counter+=1

        for word in sentence:
            for subword in word.split():
                if subword.strip():
                    if subword not in vocab:
                        vocab[subword] = 1
                    else:
                        vocab[subword] += 1

    # Adding count of rare words to the count of <unk>
    for k in list(vocab.keys())[1:]:
        if vocab[k] < threshold:
            vocab['<unk>'] += vocab[k]
            del vocab[k]

    # Sort vocab by count in descending order
    sorted_vocab = {k: v for k, v in sorted(vocab.items(), key=lambda item: item[1], reverse=True)}

    # Push <unk> to the top
    unk_val = sorted_vocab.pop('<unk>')
    sorted_vocab = {'<unk>': unk_val, **sorted_vocab}

    with open('vocab.txt', 'w') as f:
        format_str = ''
        for index, (key, count) in enumerate(sorted_vocab.items()):
            format_str += key + '\t' + str(index) + '\t' + str(count) + '\n'
        f.write(format_str)

    #print("Threshold =", threshold)

    return sorted_vocab, counter

vocab, counter = generate_vocab("C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/train2.json")
print("Overall size of my vocabulary =", len(vocab))
print("No. of times '<unk>' occurs in my vocabulary =", vocab['<unk>'])
print("No. of sentence =", counter)

Overall size of my vocabulary = 1049
No. of times '<unk>' occurs in my vocabulary = 1680
No. of sentence = 400


In [3]:
def load_training_data(file_path):
    with open(file_path, 'r') as json_file:
        return json.load(json_file)

#transition holo NNP to VBP kotobar, emission holo "Book" word ta kotobar NNP r kotobar VBP
#unique state holo distinct tag gulo niye banano list
def process_training_data(data, vocab):
    transition_probabilities = {}
    emission_probabilities = {}
    unique_states = []

    for entry in data:
        sentence_tokens = entry['sentence']
        labels = entry['labels']
        prior_state = 'head'  

        for word, tag in zip(sentence_tokens, labels):
            if word not in vocab.keys():
                word = '<unk>'

            # Update unique states
            if tag not in unique_states:
                unique_states.append(tag)

            # Update transition count
            transition_key = (prior_state, tag)
            if transition_key not in transition_probabilities:
                transition_probabilities[transition_key] = 1
            else:
                transition_probabilities[transition_key] += 1

            # Update emission count
            emission_key = (tag, word)
            if emission_key not in emission_probabilities:
                emission_probabilities[emission_key] = 1
            else:
                emission_probabilities[emission_key] += 1

            prior_state = tag

    return transition_probabilities, emission_probabilities, unique_states

def normalize_probabilities(probabilities):
    normalized_probabilities = {}
    for key in probabilities:
        state, value = key
        total = sum(v for k, v in probabilities.items() if k[0] == state)
        normalized_probabilities[key] = probabilities[key] / total
    return normalized_probabilities

#transition r emission gulo k dictionary er moto kore dekhano hoyese
def save_hmm_model(transition_probs, emission_probs, output_file):
    
    # Convert tuple keys to strings
    transition_probs = {str(k): v for k, v in transition_probs.items()}
    emission_probs = {str(k): v for k, v in emission_probs.items()}

    hmm_model = {
        'transition': transition_probs,
        'emission': emission_probs,
    }

    with open(output_file, 'w') as f:
        json.dump(hmm_model, f)

    return transition_probs, emission_probs

training_data = load_training_data('C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/train2.json')
transition_probs, emission_probs, unique_states = process_training_data(training_data, vocab)
normalized_transition_probs = normalize_probabilities(transition_probs)
normalized_emission_probs = normalize_probabilities(emission_probs)
transition_param_dict, emission_param_dict = save_hmm_model(normalized_transition_probs, normalized_emission_probs, 'hmm.json')

print("No. of transition parameters =", len(transition_param_dict))
print("No. of emission parameters =", len(emission_param_dict))

No. of transition parameters = 610
No. of emission parameters = 1254


In [4]:
import nltk

In [5]:
user_text = input("Enter your sentence: ")

Enter your sentence: Bangladesh is a country located in South Asia, bordered by India and Myanmar. It is the eighth-most populous country in the world, with a population of over 160 million people. The capital and largest city is Dhaka.


In [6]:
print(user_text)

Bangladesh is a country located in South Asia, bordered by India and Myanmar. It is the eighth-most populous country in the world, with a population of over 160 million people. The capital and largest city is Dhaka.


In [7]:
#test file theke data read kore r format kore

def format_data(user_text):
    text = user_text
    sentence_tokens = nltk.word_tokenize(text)
    #print(text)
    return sentence_tokens

#POS tag kore each word er r accuracy dey
def greedy_decoding(formatted_data, vocab, transition_param_dict, emission_param_dict, state_track):
    accuracy = []

    prior_st = 'head'
    for word in formatted_data:
        if word not in vocab.keys():
            word = '<unk>'

        max_prob = 0
        max_st = None

        for st_ in state_track:
            tr = transition_param_dict.get(str((prior_st, st_)), 1e-7)
            em = emission_param_dict.get(str((st_, word)), 0)
            prob = tr * em

            if prob > max_prob:
                max_prob = prob
                max_st = st_

        #accuracy.append(st == max_st)
        '''prior_st = max_st
        print(prior_st)'''
        
        print(max_st)
        print('Transition_prob = ',transition_param_dict.get(str((prior_st, max_st)), 1e-7))
        print('Emission_prob = ', emission_param_dict.get(str((max_st, word)), 0))
        prior_st = max_st

    return #sum(accuracy) / len(accuracy)

formatted_dev_data = format_data(user_text)
print(formatted_dev_data)
greedy_decoding(formatted_dev_data, vocab, transition_param_dict, emission_param_dict, unique_states)
#print(emission_param_dict.get(str(('VB', 'book')), 0))
#accuracy = greedy_decoding(formatted_dev_data, vocab, transition_param_dict, emission_param_dict, unique_states)
#print("Accuracy on dev data with greedy decoding =", accuracy)

['Bangladesh', 'is', 'a', 'country', 'located', 'in', 'South', 'Asia', ',', 'bordered', 'by', 'India', 'and', 'Myanmar', '.', 'It', 'is', 'the', 'eighth-most', 'populous', 'country', 'in', 'the', 'world', ',', 'with', 'a', 'population', 'of', 'over', '160', 'million', 'people', '.', 'The', 'capital', 'and', 'largest', 'city', 'is', 'Dhaka', '.']
NNP
Transition_prob =  0.2425
Emission_prob =  0.3032069970845481
VBZ
Transition_prob =  0.028182701652089408
Emission_prob =  0.24056603773584906
DT
Transition_prob =  0.14622641509433962
Emission_prob =  0.23862238622386223
NN
Transition_prob =  0.5166051660516605
Emission_prob =  0.0015503875968992248
NN
Transition_prob =  0.11636927851047324
Emission_prob =  0.25813953488372093
IN
Transition_prob =  0.2684251357641583
Emission_prob =  0.16601941747572815
NNP
Transition_prob =  0.16034985422740525
Emission_prob =  0.3032069970845481
NNP
Transition_prob =  0.3566569484936832
Emission_prob =  0.3032069970845481
,
Transition_prob =  0.180758017

In [8]:
def reverse_graph(G):
    '''Return the reversed graph g[dst][src]=G[src][dst]'''
    g = {}
    for src in G.keys():
        for dst in G[src].keys():
            if dst not in g.keys():
                g[dst] = {}
            g[dst][src] = G[src][dst]
    return g


def build_max(rg, root):
    '''Find the max in-edge for every node except for the root.'''
    mg = {}
    for dst in rg.keys():
        if dst == root:
            continue
        max_ind = -100
        max_value = -100
        for src in rg[dst].keys():
            if rg[dst][src] >= max_value:
                max_ind = src
                max_value = rg[dst][src]
        mg[dst] = {max_ind: max_value}
    return mg


def find_circle(mg):
    '''Return the first circle if find, otherwise return None'''

    for start in mg.keys():
        visited = []
        stack = [start]
        while stack:
            n = stack.pop()
            if n in visited:
                C = []
                while n not in C:
                    C.append(n)
                    n = list(mg[n].keys())[0]
                return C
            visited.append(n)
            if n in mg.keys():
                stack.extend(list(mg[n].keys()))
    return None


def chu_liu_edmond(G, root):
    ''' G: dict of dict of weights
            G[i][j] = w means the edge from node i to node j has weight w.
        root: the root node, has outgoing edges only.
    '''
    # reversed graph rg[dst][src] = G[src][dst]
    rg = reverse_graph(G)
    # root er only out edge
    rg[root] = {}
    # the maximum edge select korlam for each node other than root
    mg = build_max(rg, root)

    # check if mg is a tree (contains a circle)
    C = find_circle(mg)
    # if there is no circle, it means mg is what we want
    if not C:
        return reverse_graph(mg)

    # jesob node circle kore tader k niye compact node korlm
    all_nodes = G.keys()
    vc = max(all_nodes) + 1

    # new graph holo G_prime
    V_prime = list(set(all_nodes) - set(C)) + [vc]
    G_prime = {}
    vc_in_idx = {}
    vc_out_idx = {}
    # Now add the edges to G_prime
    for u in all_nodes:
        for v in G[u].keys():
            # incoming edge er weight calculation
            if (u not in C) and (v in C):
                if u not in G_prime.keys():
                    G_prime[u] = {}
                w = G[u][v] - list(mg[v].values())[0]
                if (vc not in G_prime[u]) or (vc in G_prime[u] and w > G_prime[u][vc]):
                    G_prime[u][vc] = w
                    vc_in_idx[u] = v

            # outgoing edge er weight calculation
            elif (u in C) and (v not in C):
                if vc not in G_prime.keys():
                    G_prime[vc] = {}
                w = G[u][v]
                if (v not in G_prime[vc]) or (v in G_prime[vc] and w > G_prime[vc][v]):
                    G_prime[vc][v] = w
                    vc_out_idx[v] = u

            # Third case: if the source and dest are all not in the circle, then just add the edge to the new graph.
            elif (u not in C) and (v not in C):
                if u not in G_prime.keys():
                    G_prime[u] = {}
                G_prime[u][v] = G[u][v]

    # Recursively run the algorihtm on the new graph G_prime
    A = chu_liu_edmond(G_prime, root)
    # print(A)

    # compacted node k vangbo, erpor incoming r outgoing edge gulo identify krbo
    # always max ta choose krbo r bakigulo delete krbo
    all_nodes_A = list(A.keys())
    for src in all_nodes_A:
        # The number of out-edges varies, could be 0 or any number <=|V\C|
        if src == vc:
            for node_in in A[src].keys():
                orig_out = vc_out_idx[node_in]
                if orig_out not in A.keys():
                    A[orig_out] = {}
                A[orig_out][node_in] = G[orig_out][node_in]
        else:
            #for dst in A[src]:
            for dst in list(A[src].keys()):
                # There must be only one in-edge to vc.
                if dst == vc:
                    orig_in = vc_in_idx[src]
                    A[src][orig_in] = G[src][orig_in]
                    del A[src][dst]
    del A[vc]


    for node in C:
        if node != orig_in:
            src = list(mg[node].keys())[0]
            if src not in A.keys():
                A[src] = {}
            A[src][node] = mg[node][src]

    return A


In [15]:
def get_edges(graph):
    edges = []
    for node in graph:
        for neighbor in graph[node]:
            # Avoid adding duplicate edges
            # if (neighbor, node) not in edges:
            edges.append((node, neighbor))
    return edges

In [17]:
'''
G = {
    0: {1: 9, 2: 10, 3: 9},  # Vertex 0 has edges to vertices 1, 2 and 3 with weights 9, 10, 9
    1: {2: 20, 3: 3},
    2: {1: 30, 3: 30},
    3: {1: 11, 2: 0}
}
print(chu_liu_edmond(G, 0))
'''

'\nG = {\n    0: {1: 9, 2: 10, 3: 9},  # Vertex 0 has edges to vertices 1, 2 and 3 with weights 9, 10, 9\n    1: {2: 20, 3: 3},\n    2: {1: 30, 3: 30},\n    3: {1: 11, 2: 0}\n}\nprint(chu_liu_edmond(G, 0))\n'

In [16]:
sen = ['I', 'go', 'to', 'school']
dp={}
G3= {0: {1: 0.13175059355956434, 2: 0.18211908429298618*10, 3: 1.0444370537508816, 4: 0.03227769819785418},
    1: {2: 0.18211908429298618, 3: 1.0444370537508816, 4: 0.03227769819785418},
    2: {1: 0.13175059355956434*3, 3: 1.0444370537508816*5, 4: 0.03227769819785418*2.5},
    3: {1: 0.13175059355956434, 2: 0.18211908429298618, 4: 0.03227769819785418*3},
    4: {1: 0.13175059355956434*2, 2: 0.18211908429298618*5, 3: 1.0444370537508816}}
#print("G3 :", G3[1][2])
dp = chu_liu_edmond(G3, 0)
print(dp)
edges = get_edges(dp)
print("Edges of the graph:", edges)
for i in edges:
    #print(i)
    x = i[0]-1
    y = i[1]-1
    #print(x)
    if x == -1:
        print('root', '-->', sen[y])
    else:
        print(sen[x],'--> ', sen[y])

{2: {1: 0.39525178067869304, 3: 5.222185268754408}, 0: {2: 1.8211908429298618}, 3: {4: 0.09683309459356254}}
Edges of the graph: [(2, 1), (2, 3), (0, 2), (3, 4)]
go -->  I
go -->  to
root --> go
to -->  school


In [10]:
print(formatted_dev_data[0])

Bangladesh


In [11]:
print(transition_param_dict)

{"('head', 'IN')": 0.115, "('IN', 'DT')": 0.293488824101069, "('DT', 'NN')": 0.5166051660516605, "('NN', ',')": 0.1078355314197052, "(',', 'DT')": 0.13008130081300814, "('DT', 'NNS')": 0.0959409594095941, "('NNS', 'VBD')": 0.06317411402157165, "('VBD', ',')": 0.023809523809523808, "(',', '``')": 0.022357723577235773, "('``', 'PRP')": 0.20224719101123595, "('PRP', 'VBP')": 0.24102564102564103, "('VBP', 'PDT')": 0.006060606060606061, "('PDT', 'DT')": 1.0, "('NNS', '.')": 0.1386748844375963, '(\'.\', "\'\'")': 0.875, "('head', 'JJ')": 0.0325, "('JJ', 'NNS')": 0.28807947019867547, "('NNS', 'VBP')": 0.10477657935285054, "('VBP', 'JJ')": 0.13333333333333333, "('JJ', 'IN')": 0.054635761589403975, "('IN', 'NNP')": 0.16034985422740525, "('NNP', 'NNP')": 0.3566569484936832, "('NNP', '.')": 0.05053449951409135, "('head', '``')": 0.085, "('PRP', 'VBD')": 0.3230769230769231, "('VBD', 'DT')": 0.20833333333333334, "('NN', 'IN')": 0.2684251357641583, "('IN', 'NN')": 0.10204081632653061, '(\',\', "\'\'

In [12]:
print(emission_param_dict)

{"('IN', 'On')": 0.002912621359223301, "('DT', 'the')": 0.4956949569495695, "('NN', 'back')": 0.0007751937984496124, "(',', ',')": 1.0, "('NNS', '<unk>')": 0.27889060092449924, "('VBD', 'read')": 0.005952380952380952, "('``', '``')": 0.9887640449438202, "('PRP', 'We')": 0.041025641025641026, "('VBP', 'have')": 0.16363636363636364, "('PDT', 'all')": 1.0, "('.', '.')": 0.9896907216494846, '("\'\'", "\'\'")': 0.9883720930232558, "('JJ', 'Many')": 0.0033112582781456954, "('VBP', 'are')": 0.23030303030303031, "('JJ', 'angry')": 0.0033112582781456954, "('IN', 'at')": 0.03300970873786408, "('NNP', 'Mrs.')": 0.004859086491739553, "('NNP', 'Yeargin')": 0.0029154518950437317, "('PRP', '<unk>')": 0.020512820512820513, "('VBD', 'did')": 0.026785714285714284, "('DT', 'a')": 0.23862238622386223, "('NN', 'lot')": 0.002325581395348837, "('IN', 'of')": 0.2320388349514563, "('NN', '<unk>')": 0.25813953488372093, "('VBZ', 'says')": 0.11320754716981132, "('NNP', '<unk>')": 0.3032069970845481, "('WP', 'who

In [13]:
print(unique_states)

['IN', 'DT', 'NN', ',', 'NNS', 'VBD', '``', 'PRP', 'VBP', 'PDT', '.', "''", 'JJ', 'NNP', 'VBZ', 'WP', 'VBN', 'RB', 'CC', 'VBG', 'RP', 'EX', 'MD', 'VB', 'RBR', 'TO', 'CD', 'PRP$', ':', 'NNPS', 'JJR', 'POS', '-LRB-', '-RRB-', 'WDT', 'WRB', 'RBS', 'JJS', '$', 'WP$', 'SYM', 'LS']
